In [13]:
from helperutil import file_date_util as util
import sys
import os
sys.path.insert(0, './dags')

from restaurant_pipeline import default_args
from restaurant_pipeline import get_process_file_list

In [6]:
import pandas as pd
from datetime import date as d

In [4]:
from sqlalchemy import create_engine

In [77]:
# # setup default arguments
# default_args = {
#     'download_url': 'https://data.cityofnewyork.us/resource/9w7m-hzhe.json?inspection_date=',
#     'download_date_days_ago': -3,
#     'download_date_months_ago': -3,
#     'db_hostname': '192.168.1.221',
#     'db_username': 'restuser',
#     'db_password': 'restuser',
#     'db_database': 'restaurantdb',
# }


# setup default arguments
default_args = {
    'download_url': 'https://data.cityofnewyork.us/resource/9w7m-hzhe.json?inspection_date=',
    'download_date_days_ago': -3,
    'download_date_months_ago': -4,
    'db_hostname': 'localhost',
    'db_username': '',
    'db_password': '',
    'db_database': 'restaurant_inbox',
}



In [78]:
engine = create_engine("mysql+pymysql://" + default_args['db_username'] + ":" + default_args['db_password'] + "@" + default_args['db_hostname'] + "/" + default_args['db_database'] + "", echo=False)
conn = engine.connect()

In [26]:
files_to_process = get_process_file_list()
dataframes = (pd.read_json(f, lines=True, encoding='UTF8') for f in files_to_process)
big_dataframe = pd.concat(dataframes, ignore_index=True)
# big_dataframe = big_dataframe.reset_index()

/home/developer/python-dev/airflow_project1/env/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


In [27]:
# downloading_list
# path_name

# date_end
# date_start

big_dataframe.head(2)

,action,boro,building,camis,critical_flag,cuisine_description,dba,grade,grade_date,inspection_date,inspection_type,phone,record_date,score,street,violation_code,violation_description,zipcode
0,Violations were cited in the following area(s).,BRONX,106,50074686,Critical,"Juice, Smoothies, Fruit Salads",AMORE JUICE BAR,Not Yet Graded,None,2018-05-11T00:00:00.000,Pre-permit (Operational) / Initial Inspection,9172597091,2018-05-16T06:01:08.000,28.0,E 165TH ST,04A,Food Protection Certificate not held by superv...,10452.0
1,Violations were cited in the following area(s).,MANHATTAN,75,41466809,Critical,American,DINA RATA AT ANDAZ HOTEL,None,None,2018-05-11T00:00:00.000,Cycle Inspection / Initial Inspection,2126991692,2018-05-16T06:01:08.000,34.0,WALL STREET,06E,"Sanitized equipment or utensil, including in-u...",10005.0


In [28]:
big_dataframe = big_dataframe[['camis', 'grade', 'grade_date']].drop_duplicates(subset=['camis', 'grade', 'grade_date'])
big_dataframe = big_dataframe[big_dataframe['grade_date'].isna() == False]
big_dataframe.columns = ['business_number', 'grade', 'grade_date']


In [66]:
df_grade = big_dataframe

In [67]:
df_grade['grade_date'] = df_grade['grade_date'].astype('datetime64')
# s[s.isna() == True].value_counts()
# s.value_counts

In [72]:
# df_grade.head(10)
df_grade = df_grade[df_grade['grade_date'].isna() == False]

In [64]:
#df_grade.head(10)
df_grade = df_grade[df_grade['grade_date'].isna() == False]

In [75]:
df_grade[df_grade.isna() == False].head(10)

,business_number,grade,grade_date
2,41688097,A,2018-05-11
3,50040458,A,2018-05-11
4,40866029,A,2018-05-11
5,41244032,A,2018-05-11
6,50075060,A,2018-05-11
13,41688121,A,2018-05-11
21,50013521,A,2018-05-11
22,41688120,A,2018-05-11
23,41123042,A,2018-05-11
25,41688112,A,2018-05-11


In [79]:
truncate_sql = """
truncate table trans_dim_grade
"""
update_res = pd.io.sql.execute(truncate_sql, conn)

In [80]:
df_grade.to_sql('trans_dim_grade', con=engine, if_exists='append', index=False)

In [82]:
# truncate_sql = """
# truncate table dim_grade
# """
# update_res = pd.io.sql.execute(truncate_sql, conn)

In [83]:
insert_sql = """
insert into dim_grade (business_number, grade, grade_date, create_date, modify_date)
select business_number, grade, grade_date, now(), now() from trans_dim_grade
"""
update_res = pd.io.sql.execute(insert_sql, conn)